In [ ]:
import pandas as pd
import requests
import geohash2
import configparser


pd.set_option('display.max_columns', None)

In [ ]:
config = configparser.ConfigParser()
config.read("../datas.ini")
datas = config["kakao"]

In [138]:
class SearchDatas:
    
    def __init__(self, query):
        self.query = query
        self.zigbang_df = None
        self.kakao_df = None
        
    def zigbang(self, precision, sale = "전세|월세", service = "원룸"):
        url = f"https://apis.zigbang.com/search?q={self.query}"
        response = requests.get(url)
        
        lat, lng = response.json()['items'][0]['lat'], response.json()['items'][0]['lng']
        
        geohash = geohash2.encode(lat, lng, precision=precision)
        url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash={geohash}&rent_gteq=0&sales_type_in={sale}&service_type_eq={service}"
        response = requests.get(url)
        
        items = response.json()['items']
        ids = [item['item_id'] for item in items]
        
        url = "https://apis.zigbang.com/v2/items/list"

        dfs = []
        print(len(ids))
        for idx in range(0, len(ids), 900):
            start, end = idx, idx + 900

            params = {"domain": "zigbang", "withCoalition": "false", "item_ids": ids[start:end],}

            response = requests.post(url, params)

            datas = response.json()['items']
            df = pd.DataFrame(datas)

            df['lat'] =  df['random_location'].apply(lambda x: x['lat'])
            df['lng'] =  df['random_location'].apply(lambda x: x['lng'])

            df['공급면적_m2'] = df['공급면적'].apply(lambda x:x['m2'])
            df['공급면적_p'] = df['공급면적'].apply(lambda x:x['p'])
            df['전용면적_m2'] = df['전용면적'].apply(lambda x:x['m2'])
            df['전용면적_p'] = df['전용면적'].apply(lambda x:x['p'])

            df.drop(columns=["random_location", "공급면적", "전용면적"], inplace=True)
            dfs.append(df)

        self.zigbang_df = pd.concat(dfs)
        self.zigbang_df.reset_index(drop=True, inplace=True)
        print("end crawling")

    
    def kakao(self, REST_API_KEY):
        headers = {"Authorization": f"KakaoAK {REST_API_KEY}",}
        
        cate_ls =  ["MT1", "CS2", "PS3", "SC4", "AC5", "PK6", "OL7", "SW8", "BK9", "CT1", "AG2", "PO3", "AT4",
                    "AD5", "FD6", "CE7", "HP8", "PM9"]
        
        page, size, radius = 1, 15, 200
        dfs_2 = []
        self.zigbang_df[cate_ls] = None
        for idx, loc in enumerate(self.zigbang_df[['lat', 'lng']].values[:2]):
            print(idx)
            lat, lng = loc
            for category in cate_ls:
                while True:
                    url = f"https://dapi.kakao.com/v2/local/search/category.json?category_group_code={category}&page={page}&size={size}&sort=distance&x={lng}&y={lat}&radius={radius}"
                    response = requests.get(url, headers=headers)

                    datas = response.json()['documents']

                    df = pd.DataFrame(datas)
                    df['매물_lat'] = lat
                    df['매물_lng'] = lng

                    dfs_2.append(df)

                    if response.json()['meta']['is_end']:
                        self.zigbang_df.loc[idx, category] = response.json()['meta']['total_count']
                        break

                    page += 1

        self.kakao_df = pd.concat(dfs_2)
        self.kakao_df.reset_index(drop=True, inplace=True)
        print("end crawling")
        

In [139]:
test1 = SearchDatas("서울시 광진구")

In [140]:
test1.zigbang(precision=5)

2196
end crawling


In [122]:
test1.zigbang_df

,section_type,item_id,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,계약면적,room_type_title,floor,floor_string,building_floor,title,is_first_movein,room_type,address,is_zzim,status,service_type,tags,address1,address2,address3,manage_cost,reg_date,is_new,lat,lng,공급면적_m2,공급면적_p,전용면적_m2,전용면적_p
0,None,27052098,https://ic.zigbang.com/ic/items/27052098/1.jpg,전세,전세,30000,0,46.28,None,None,5,5,5,공간이넓은 주차가능한 투룸,None,04,광진구 자양동,False,True,빌라,[추천],서울시 광진구 자양동,None,None,2,2021-04-28T14:11:43+09:00,False,37.529062,127.081526,46.28,14,42.98,13
1,None,27359986,https://ic.zigbang.com/ic/items/27359986/1.jpg,월세,월세,1000,40,49.59,None,None,반지하,반지하,5,완전 올수리된 투룸,None,04,광진구 자양동,False,True,빌라,[],서울시 광진구 자양동,None,None,1,2021-05-22T15:51:33+09:00,False,37.529678,127.079510,49.59,15,39.67,12
2,None,27392347,https://ic.zigbang.com/ic/items/27392347/1.jpg,전세,전세,24000,0,29.75,None,None,3,3,5,[신축첫입주]원룸원거실/9평/방窓2/東향최고채광/중문有,None,02,광진구 자양동,False,True,원룸,[],서울시 광진구 자양동,None,None,7,2021-05-27T18:53:34+09:00,True,37.529785,127.084359,29.75,9,29.75,9
3,None,27425148,https://ic.zigbang.com/ic/items/27425148/1.jpg,전세,전세,35000,0,56.20,None,None,3,3,5,전용14평신축/3bay구조/풀옵션/광폭거실/南東향/중문有,None,04,광진구 자양동,False,True,빌라,[],서울시 광진구 자양동,None,None,7,2021-05-27T17:55:53+09:00,True,37.529305,127.084363,56.20,17,46.28,14
4,None,27045859,https://ic.zigbang.com/ic/items/27045859/1.jpg,월세,월세,1000,50,35.09,None,None,3,3,4,♥ 자양2동/투룸/남향/올수리 ♥,None,04,광진구 자양동,False,True,빌라,[],서울시 광진구 자양동,None,None,0,2021-04-29T15:14:00+09:00,False,37.529418,127.083117,35.09,10.6,34.01,10.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2191,None,27257341,https://ic.zigbang.com/ic/items/27257341/1.jpg,월세,월세,2000,70,46.28,None,None,3,3,3,★면목4동 구민회관 인근. 내부 수리한집. 창고.옥상!,None,04,중랑구 면목동,False,True,빌라,[추천],서울시 중랑구 면목동,None,None,0,2021-05-21T15:38:28+09:00,False,37.572523,127.083900,46.28,14,44.63,13.5
2192,None,27304805,https://ic.zigbang.com/ic/items/27304805/1.jpg,월세,월세,5000,50,34.00,None,None,3,3,4,🟦용마산역10분🟦중랑천산책로 인근🟦중곡초🟦,None,04,중랑구 면목동,False,True,빌라,[],서울시 중랑구 면목동,None,None,2,2021-05-18T13:31:02+09:00,False,37.573249,127.081191,34.00,10.3,33.00,10
2193,None,27403047,https://ic.zigbang.com/ic/items/27403047/1.jpg,월세,월세,500,40,36.36,None,None,반지하,반지하,2,용마산역반지하층방2칸,None,04,중랑구 면목동,False,True,빌라,[],서울시 중랑구 면목동,None,None,0,2021-05-26T11:30:09+09:00,True,37.572923,127.082503,36.36,11,33.06,10
2194,None,27416196,https://ic.zigbang.com/ic/items/27416196/1.jpg,월세,월세,500,40,39.67,None,None,반지하,반지하,2,용마산역8분 습기안차는 투룸 독채반지하,None,04,중랑구 면목동,False,True,빌라,[],서울시 중랑구 면목동,None,None,0,2021-05-27T10:56:01+09:00,True,37.573425,127.082497,39.67,12,33.06,10


In [141]:
test1.kakao(REST_API_KEY=datas['REST_API'])

0
1
end crawling


In [144]:
test1.zigbang_df.head(2)

,section_type,item_id,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,계약면적,room_type_title,floor,floor_string,building_floor,title,is_first_movein,room_type,address,is_zzim,status,service_type,tags,address1,address2,address3,manage_cost,reg_date,is_new,lat,lng,공급면적_m2,공급면적_p,전용면적_m2,전용면적_p,MT1,CS2,PS3,SC4,AC5,PK6,OL7,SW8,BK9,CT1,AG2,PO3,AT4,AD5,FD6,CE7,HP8,PM9
0,None,27052098,https://ic.zigbang.com/ic/items/27052098/1.jpg,전세,전세,30000,0,46.28,None,None,5,5,5,공간이넓은 주차가능한 투룸,None,04,광진구 자양동,False,True,빌라,[추천],서울시 광진구 자양동,None,None,2,2021-04-28T14:11:43+09:00,False,37.529086,127.081573,46.28,14,42.98,13,0,6,8,0,19,1,0,0,1,0,21,0,0,0,18,6,2,1
1,None,27359986,https://ic.zigbang.com/ic/items/27359986/1.jpg,월세,월세,1000,40,49.59,None,None,반지하,반지하,5,완전 올수리된 투룸,None,04,광진구 자양동,False,True,빌라,[],서울시 광진구 자양동,None,None,1,2021-05-22T15:51:33+09:00,False,37.529671,127.079497,49.59,15,39.67,12,0,6,4,0,10,1,0,0,0,0,13,0,0,0,26,5,0,0


In [142]:
test1.kakao_df.tail()

,매물_lat,매물_lng,address_name,category_group_code,category_group_name,category_name,distance,id,phone,place_name,place_url,road_address_name,x,y
98,37.529671,127.079497,서울 광진구 자양동 639-5,CE7,카페,음식점 > 카페,74,1729653532,,퍼스트플로어,http://place.map.kakao.com/1729653532,서울 광진구 뚝섬로52길 41,127.078918538391,37.5301561472225
99,37.529671,127.079497,서울 광진구 자양동 648-26,CE7,카페,음식점 > 카페,113,523864595,,이월오일,http://place.map.kakao.com/523864595,서울 광진구 뚝섬로52라길 39,127.080743770403,37.529443122378
100,37.529671,127.079497,서울 광진구 자양동 638,CE7,카페,음식점 > 카페 > 커피전문점 > 컴포즈커피,124,1008530191,02-458-2562,컴포즈커피 광진자양점,http://place.map.kakao.com/1008530191,서울 광진구 뚝섬로52나길 2,127.0791839220463,37.53076865043087
101,37.529671,127.079497,서울 광진구 자양동 646-46,CE7,카페,음식점 > 카페,180,148658227,,클라씩 하우스,http://place.map.kakao.com/148658227,서울 광진구 뚝섬로52마길 49,127.081195583464,37.528776071551
102,37.529671,127.079497,서울 광진구 자양동 637,CE7,카페,음식점 > 카페 > 테마카페 > 디저트카페,182,2134004473,010-3215-3666,에클레시아,http://place.map.kakao.com/2134004473,서울 광진구 뚝섬로52길 13,127.079431135532,37.5313126891155


In [61]:
import geohash2

pd.set_option('display.max_columns', None)

In [3]:
addr = "서울시 광진구"
url = "https://apis.zigbang.com/search?q={}".format(addr)
print(url)

https://apis.zigbang.com/search?q=서울시 광진구


In [4]:
response = requests.get(url)
response

<Response [200]>

In [11]:
lat, lng = response.json()['items'][0]['lat'], response.json()['items'][0]['lng']
print(lat, lng)

37.5386161804199 127.082374572754


In [13]:
geohash = geohash2.encode(lat, lng, precision=5)

url = "https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash={}&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸".format(geohash)
print(url)

https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash=wydme&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸


In [14]:
response = requests.get(url)

In [19]:
items = response.json()['items']
ids = [item['item_id'] for item in items]
print(len(items))

2193


In [69]:
url = "https://apis.zigbang.com/v2/items/list"

dfs = []
print(len(ids))
for idx in range(0, len(ids), 900):
    start, end = idx, idx + 900
    
    params = {"domain": "zigbang", "withCoalition": "false", "item_ids": ids[start:end],}
    
    response = requests.post(url, params)
    
    datas = response.json()['items']
    df = pd.DataFrame(datas)
    
    df['lat'] =  df['random_location'].apply(lambda x: x['lat'])
    df['lng'] =  df['random_location'].apply(lambda x: x['lng'])
    
    df['공급면적_m2'] = df['공급면적'].apply(lambda x:x['m2'])
    df['공급면적_p'] = df['공급면적'].apply(lambda x:x['p'])
    df['전용면적_m2'] = df['전용면적'].apply(lambda x:x['m2'])
    df['전용면적_p'] = df['전용면적'].apply(lambda x:x['p'])
    
    df.drop(columns=["random_location", "공급면적", "전용면적"], inplace=True)
    dfs.append(df)
    
zigbang_df = pd.concat(dfs)
zigbang_df.reset_index(drop=True, inplace=True)

2193


In [72]:
print(zigbang_df.shape)
zigbang_df.tail()

,section_type,item_id,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,계약면적,room_type_title,floor,floor_string,building_floor,title,is_first_movein,room_type,address,is_zzim,status,service_type,tags,address1,address2,address3,manage_cost,reg_date,is_new,lat,lng,공급면적_m2,공급면적_p,전용면적_m2,전용면적_p
0,None,27052098,https://ic.zigbang.com/ic/items/27052098/1.jpg,전세,전세,30000,0,46.28,None,None,5,5,5,공간이넓은 주차가능한 투룸,None,04,광진구 자양동,False,True,빌라,[추천],서울시 광진구 자양동,None,None,2,2021-04-28T14:11:43+09:00,False,37.529094,127.080911,46.28,14,42.98,13
1,None,27359986,https://ic.zigbang.com/ic/items/27359986/1.jpg,월세,월세,1000,40,49.59,None,None,반지하,반지하,5,완전 올수리된 투룸,None,04,광진구 자양동,False,True,빌라,[],서울시 광진구 자양동,None,None,1,2021-05-22T15:51:33+09:00,False,37.529662,127.079536,49.59,15,39.67,12
2,None,27392347,https://ic.zigbang.com/ic/items/27392347/1.jpg,전세,전세,24000,0,29.75,None,None,3,3,5,[신축첫입주]원룸원거실/9평/방窓2/東향최고채광/중문有,None,02,광진구 자양동,False,True,원룸,[],서울시 광진구 자양동,None,None,7,2021-05-27T18:53:34+09:00,True,37.529781,127.084364,29.75,9,29.75,9
3,None,27425148,https://ic.zigbang.com/ic/items/27425148/1.jpg,전세,전세,35000,0,56.20,None,None,3,3,5,전용14평신축/3bay구조/풀옵션/광폭거실/南東향/중문有,None,04,광진구 자양동,False,True,빌라,[],서울시 광진구 자양동,None,None,7,2021-05-27T17:55:53+09:00,True,37.529775,127.084334,56.20,17,46.28,14
4,None,27045859,https://ic.zigbang.com/ic/items/27045859/1.jpg,월세,월세,1000,50,35.09,None,None,3,3,4,♥ 자양2동/투룸/남향/올수리 ♥,None,04,광진구 자양동,False,True,빌라,[],서울시 광진구 자양동,None,None,0,2021-04-29T15:14:00+09:00,False,37.529439,127.082489,35.09,10.6,34.01,10.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2188,None,27257341,https://ic.zigbang.com/ic/items/27257341/1.jpg,월세,월세,2000,70,46.28,None,None,3,3,3,★면목4동 구민회관 인근. 내부 수리한집. 창고.옥상!,None,04,중랑구 면목동,False,True,빌라,[추천],서울시 중랑구 면목동,None,None,0,2021-05-21T15:38:28+09:00,False,37.573015,127.084535,46.28,14,44.63,13.5
2189,None,27304805,https://ic.zigbang.com/ic/items/27304805/1.jpg,월세,월세,5000,50,34.00,None,None,3,3,4,🟦용마산역10분🟦중랑천산책로 인근🟦중곡초🟦,None,04,중랑구 면목동,False,True,빌라,[],서울시 중랑구 면목동,None,None,2,2021-05-18T13:31:02+09:00,False,37.572740,127.081811,34.00,10.3,33.00,10
2190,None,27403047,https://ic.zigbang.com/ic/items/27403047/1.jpg,월세,월세,500,40,36.36,None,None,반지하,반지하,2,용마산역반지하층방2칸,None,04,중랑구 면목동,False,True,빌라,[],서울시 중랑구 면목동,None,None,0,2021-05-26T11:30:09+09:00,True,37.573426,127.082533,36.36,11,33.06,10
2191,None,27416196,https://ic.zigbang.com/ic/items/27416196/1.jpg,월세,월세,500,40,39.67,None,None,반지하,반지하,2,용마산역8분 습기안차는 투룸 독채반지하,None,04,중랑구 면목동,False,True,빌라,[],서울시 중랑구 면목동,None,None,0,2021-05-27T10:56:01+09:00,True,37.572943,127.082500,39.67,12,33.06,10


In [123]:
REST_API_KEY = datas['REST_API']

In [74]:
headers = {"Authorization": f"KakaoAK {REST_API_KEY}",}

In [ ]:
cate_ls =  ["MT1", "CS2", "PS3", "SC4", "AC5", "PK6", "OL7", "SW8", "BK9", "CT1", "AG2", "PO3", "AT4",
"AD5", "FD6", "CE7", "HP8", "PM9"]

In [112]:
page, size, radius = 1, 15, 200
dfs_2 = []
zigbang_df[category] = None
for idx, loc in enumerate(zigbang_df[['lat', 'lng']].values):
    print(idx)
    lat, lng = loc
    for category in cate_ls:
        while True:
            url = f"https://dapi.kakao.com/v2/local/search/category.json?category_group_code={category}&page={page}&size={size}&sort=distance&x={lng}&y={lat}&radius={radius}"
            response = requests.get(url, headers=headers)
            
            datas = response.json()['documents']
            
            df = pd.DataFrame(datas)
            df['매물_lat'] = lat
            df['매물_lng'] = lng
            
            dfs_2.append(df)
            
            if response.json()['meta']['is_end']:
#                 print(idx)
#                 print(category)
#                 print("총 개수 :", response.json()['meta']['total_count'])
                zigbang_df.loc[idx, category] = response.json()['meta']['total_count']
                break
                
            page += 1
            
kakao_df = pd.concat(dfs_2)

0
MT1
총 개수 : 0
0
CS2
총 개수 : 6
0
PS3
총 개수 : 7
0
SC4
총 개수 : 0
0
AC5
총 개수 : 16
0
PK6
총 개수 : 1
0
OL7
총 개수 : 0
0
SW8
총 개수 : 0
0
BK9
총 개수 : 1
0
CT1
총 개수 : 0
0
AG2
총 개수 : 17
0
PO3
총 개수 : 0
0
AT4
총 개수 : 0
0
AD5
총 개수 : 0
0
FD6
총 개수 : 12
0
CE7
총 개수 : 5
0
HP8
총 개수 : 1
0
PM9
총 개수 : 1
1
MT1
총 개수 : 0
1
CS2
총 개수 : 6
1
PS3
총 개수 : 4
1
SC4
총 개수 : 0
1
AC5
총 개수 : 10
1
PK6
총 개수 : 1
1
OL7
총 개수 : 0
1
SW8
총 개수 : 0
1
BK9
총 개수 : 0
1
CT1
총 개수 : 0
1
AG2
총 개수 : 13
1
PO3
총 개수 : 0
1
AT4
총 개수 : 0
1
AD5
총 개수 : 0
1
FD6
총 개수 : 26
1
CE7
총 개수 : 5
1
HP8
총 개수 : 0
1
PM9
총 개수 : 0


In [113]:
print(kakao_df.shape)
kakao_df.tail()

,매물_lat,매물_lng,address_name,category_group_code,category_group_name,category_name,distance,id,phone,place_name,place_url,road_address_name,x,y
0,37.529094,127.080911,서울 광진구 자양동 648-32,CS2,편의점,"가정,생활 > 편의점 > GS25",36,698747413,1644-5425,GS25 자양타운빌점,http://place.map.kakao.com/698747413,서울 광진구 뚝섬로52라길 47,127.081234617338,37.5293004275518
1,37.529094,127.080911,서울 광진구 자양동 639-38,CS2,편의점,"가정,생활 > 편의점 > 세븐일레븐",106,1152917644,02-457-3582,세븐일레븐 자양강변점,http://place.map.kakao.com/1152917644,서울 광진구 뚝섬로54길 46,127.079963402017,37.5296833196928
2,37.529094,127.080911,서울 광진구 자양동 649-37,CS2,편의점,"가정,생활 > 편의점 > CU",126,1374874052,070-4154-0476,CU 자양진영점,http://place.map.kakao.com/1374874052,서울 광진구 뚝섬로54길 31,127.080407403054,37.5301623509356
3,37.529094,127.080911,서울 광진구 자양동 650-6,CS2,편의점,"가정,생활 > 편의점 > GS25",172,12296745,02-456-6432,GS25 자양에덴점,http://place.map.kakao.com/12296745,서울 광진구 뚝섬로56길 34,127.082601912609,37.5298797240286
4,37.529094,127.080911,서울 광진구 자양동 666-4,CS2,편의점,"가정,생활 > 편의점 > CU",179,720100624,02-455-3242,CU 자양한강점,http://place.map.kakao.com/720100624,서울 광진구 뚝섬로56길 59,127.082922081795,37.5288559620834
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,37.529662,127.079536,서울 광진구 자양동 639-5,CE7,카페,음식점 > 카페,77,1729653532,,퍼스트플로어,http://place.map.kakao.com/1729653532,서울 광진구 뚝섬로52길 41,127.078918538391,37.5301561472225
1,37.529662,127.079536,서울 광진구 자양동 648-26,CE7,카페,음식점 > 카페,109,523864595,,이월오일,http://place.map.kakao.com/523864595,서울 광진구 뚝섬로52라길 39,127.080743770403,37.529443122378
2,37.529662,127.079536,서울 광진구 자양동 638,CE7,카페,음식점 > 카페 > 커피전문점 > 컴포즈커피,126,1008530191,02-458-2562,컴포즈커피 광진자양점,http://place.map.kakao.com/1008530191,서울 광진구 뚝섬로52나길 2,127.0791839220463,37.53076865043087
3,37.529662,127.079536,서울 광진구 자양동 646-46,CE7,카페,음식점 > 카페,176,148658227,,클라씩 하우스,http://place.map.kakao.com/148658227,서울 광진구 뚝섬로52마길 49,127.081195583464,37.528776071551
